# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Import libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# Import libraries
import os
import re 
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline , FeatureUnion
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC

import pickle

In [4]:
##REMOVE###
#workspace analysis
import sklearn
print(np.__version__)
print(pd.__version__)
#print(sns.__version__)
print(sklearn.__version__)
from platform import python_version
print(python_version())

1.12.1
0.23.3
0.19.1
3.6.3


In [5]:
# load data from database
engine = create_engine('sqlite:///ETL_Preparation.db') 
df = pd.read_sql_table('ETL_Preparation_table', engine)

X = df['message']   # only column 'message' relevant
y = df.iloc[:,4:]

In [6]:
# test
print(df.head(3))
print(df.shape)
print(type(df))
#df.describe()
#Y.head(3)

   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   

   request  offer  aid_related  medical_help  medical_products      ...        \
0        0      0            0             0                 0      ...         
1        0      0            1             0                 0      ...         
2        0      0            0             0                 0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0  

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [6]:
# Test tokenize 
for message in X[:5]:
        tokens = tokenize(message)
        print(message)
        print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
# One pipeline for each classifier(variation)
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
# change pipeline number only here!
pipeline_fit = pipeline1.fit(X_train, y_train)  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
# output für pipeline1
y_pred_train = pipeline_fit.predict(X_train)
y_pred_test = pipeline_fit.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, y_pred_train)*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_test)*100)
print(" ")

# Classification report
print("--------------------------------classification report TRAIN--------------------------------")
print(classification_report(y_train, y_pred_train, target_names=y.columns.values))
print("--------------------------------classification report TEST--------------------------------")
print(classification_report(y_test, y_pred_test, target_names=y.columns.values))

The training accuracy is: 
28.1812633506
 
The test accuracy is: 
26.9606347269
 
--------------------------------classification report TRAIN--------------------------------
                        precision    recall  f1-score   support

               related       0.83      0.94      0.88     15038
               request       0.79      0.54      0.64      3333
                 offer       0.33      0.06      0.10        89
           aid_related       0.77      0.61      0.68      8130
          medical_help       0.67      0.30      0.41      1560
      medical_products       0.70      0.38      0.50      1018
     search_and_rescue       0.66      0.24      0.35       554
              security       0.53      0.09      0.15       348
              military       0.61      0.33      0.42       649
                 water       0.78      0.68      0.73      1256
                  food       0.81      0.70      0.75      2194
               shelter       0.80      0.55      0.65    

In [ ]:
##REMOVE##
# output für pipeline1
y_pred_train = pipeline_fit.predict(X_train)
y_pred_test = pipeline_fit.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, y_pred_train)*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_test)*100)
print(" ")

# Classification report
print("--------------------------------classification report TRAIN--------------------------------")
print(classification_report(y_train, y_pred_train, target_names=y.columns.values))
print("--------------------------------classification report TEST--------------------------------")
print(classification_report(y_test, y_pred_test, target_names=y.columns.values))

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
# pipeline1.get_params().keys()
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline1, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__learning_rate': [0.01, 0.02, 0.05], 'classifier__estimator__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

In [18]:
# Best result
cv.best_estimator_

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...ator=None,
          learning_rate=0.01, n_estimators=10, random_state=None),
           n_jobs=1))])

In [19]:
print("The best hyperparameters from Grid Search are:")
print(cv.best_params_)
print("")
print("The mean accuracy of model with these hyperparameters is:")
print(cv.best_score_)
cv_best_unfitted = cv.best_estimator_
cv_best = cv.best_estimator_
cv_best.fit(X_train, y_train)

The best hyperparameters from Grid Search are:
{'classifier__estimator__learning_rate': 0.01, 'classifier__estimator__n_estimators': 10}

The mean accuracy of model with these hyperparameters is:
0.547995209975


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...ator=None,
          learning_rate=0.01, n_estimators=10, random_state=None),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
# output für pipeline4
y_pred_train_cv = cv_best.predict(X_train)
y_pred_test_cv = cv_best.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, y_pred_train)*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_test)*100)
print(" ")

# Classification report
print("--------------------------------classification report TRAIN--------------------------------")
print(classification_report(y_train, y_pred_train, target_names=y.columns.values))
print("--------------------------------classification report TEST--------------------------------")
print(classification_report(y_test, y_pred_test, target_names=y.columns.values))

The training accuracy is: 
27.6116366595
 
The test accuracy is: 
26.1061946903
 
--------------------------------classification report TRAIN--------------------------------
                        precision    recall  f1-score   support

               related       0.82      0.95      0.88     15040
               request       0.80      0.55      0.65      3346
                 offer       0.44      0.10      0.16        82
           aid_related       0.77      0.60      0.68      8127
          medical_help       0.65      0.30      0.41      1568
      medical_products       0.68      0.34      0.45       970
     search_and_rescue       0.70      0.23      0.35       568
              security       0.48      0.06      0.10       352
              military       0.67      0.36      0.47       619
                 water       0.78      0.66      0.71      1249
                  food       0.82      0.70      0.76      2207
               shelter       0.80      0.54      0.64    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [34]:
# Use pipeline2 which uses a Random Forrest classifier
pipeline_fit = pipeline2.fit(X_train, y_train)  # change pipeline number only here!

In [35]:
# output für pipeline2
y_pred_train = pipeline_fit.predict(X_train)
y_pred_test = pipeline_fit.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, y_pred_train)*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_test)*100)
print(" ")

# Classification report
print("--------------------------------classification report TRAIN--------------------------------")
print(classification_report(y_train, y_pred_train, target_names=y.columns.values))
print("--------------------------------classification report TEST--------------------------------")
print(classification_report(y_test, y_pred_test, target_names=y.columns.values))

The training accuracy is: 
88.7498728512
 
The test accuracy is: 
22.0628623741
 
--------------------------------classification report TRAIN--------------------------------
                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     15038
               request       1.00      0.93      0.96      3333
                 offer       1.00      0.70      0.82        89
           aid_related       1.00      0.97      0.98      8130
          medical_help       1.00      0.85      0.92      1560
      medical_products       1.00      0.85      0.92      1018
     search_and_rescue       1.00      0.79      0.88       554
              security       1.00      0.75      0.86       348
              military       1.00      0.86      0.92       649
                 water       1.00      0.91      0.95      1256
                  food       1.00      0.94      0.97      2194
               shelter       1.00      0.91      0.95    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [21]:
# SAVE GRIDSEARCH, BEST MODEL((un)+fitted) AND IMPROVED fitted)

with open('cv_gridsearch_result.pkl', 'wb') as file:
    pickle.dump(cv, file)
    
# with open('cv_best_result_unfitted.pkl', 'wb') as file:
#     pickle.dump(cv_best_unfitted, file)

with open('cv_best_result.pkl', 'wb') as file:
    pickle.dump(cv_best, file)

with open('cv_improved_result.pkl', 'wb') as file:
    pickle.dump(pipeline_fit, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.